In [31]:
import re
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import itertools

# My Solution to Exercise 1

In [85]:
def obtain_graph(source = 'graph.txt', dtype = 'dict'):
    with open(source, 'r') as f:
        data = f.read().splitlines()
    
    nodes = {}
    G_nodes = {}
    for idx, line in enumerate(data):
        node = re.search(r'node%s' % int(idx), line).group()
        vals = re.findall(r'node\d+\s\d+\.{,1}\d+', line)
        nodes[node] = [(entry.split(' ')[0], float(entry.split(' ')[1])) for entry in vals]
        G_nodes[node] = {n: {'weight': w} for n,w in nodes[node]}
    
    if dtype == 'dict':
        return nodes
    elif dtype == 'DiGraph':
        return nx.DiGraph(G_nodes)
    else:
        raise ValueError('Return data type unsupported')
        
def draw_graph(G, lbl=True, arr=False):
    nx.draw(G, with_labels=lbl, arrows=arr)
    plt.show()

In [86]:
txt = 'graph.txt'
graph_dict = obtain_graph(txt, 'dict')
G = obtain_graph(txt, 'DiGraph')

In [101]:
target = nx.bellman_ford(G, 'node0')[1]['node99']

In [103]:
# dumb algo
n = 0
current = 'node0'
dest = 'node99'
total_cost = 0
path = [current]
J = []
if current != dest:
    J.append(10000)
while True:
    if current == dest:
        cost = 0
        total_cost += 0
        break
    else:
        keys = [key for key, val in graph_dict[current]]
        vals = [val for key, val in graph_dict[current]]
        costs = np.asarray(vals)
        total_cost += costs.min()
        current = keys[costs.argmin()]
        path.append(current)

In [104]:
total_cost, path

(427.10000000000002,
 ['node0',
  'node1',
  'node6',
  'node9',
  'node13',
  'node38',
  'node40',
  'node47',
  'node50',
  'node53',
  'node56',
  'node57',
  'node58',
  'node64',
  'node65',
  'node66',
  'node68',
  'node71',
  'node74',
  'node76',
  'node77',
  'node78',
  'node81',
  'node89',
  'node97',
  'node98',
  'node99'])